In [ ]:
# Mout Google Drive
# https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)
# %pwd %ls
# run github settings
%run /content/drive/MyDrive/CNNStanford/pytorch/pytorch_sandbox/Colab_Helper.ipynb

In [ ]:
MESSAGE = "clean file & gitignore again"
!git config --global user.email "ronyginosar@mail.huji.ac.il"
!git config --global user.name "ronyginosar"
!git add .

In [ ]:
!git commit -m "{MESSAGE}"
!git push "{GIT_PATH}"

In [ ]:
import torch
import torch.nn as nn

#%% Define SRB sub-module
class SRB(nn.Module):
    def __init__(self, channels, kernel=3, padding=1 ):
        super().__init__()
        self.conv = nn.Conv2d(channels, channels, kernel_size=(kernel,kernel), padding=padding)
        self.relu = nn.ReLU()

    def forward(self, x):
        y = self.conv(x) + x
        y = self.relu(y)
        return y


#%% Define the network
class TwoInputsNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=16, kernel=3, padding=1 ):
        super().__init__()
        self.singleSRBPath = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=(kernel,kernel), padding=padding),
            nn.ReLU(),
            SRB(out_channels)
        )
        self.jointSRB = SRB(out_channels*2)
        self.finalConv = nn.Conv2d(out_channels*2, 1, kernel_size=(kernel,kernel), padding=padding)

    def forward(self, im1, im2):
        p1 = self.singleSRBPath(im1)
        p2 = self.singleSRBPath(im2)
        z = torch.cat((p1,p2), dim=1)  # dim1 to get 32 channels in the concat
        z = self.jointSRB(z)
        z = self.finalConv(z)
        return z

#%%
"""
Test the network on random inputs: create network instance,
generate random input and run a forward pass
"""
net = TwoInputsNet()
im1 = torch.randn((1,3,480,640))
im2 = torch.randn((1,3,480,640))
out = net(im1, im2)
print(out.shape)